<a href="https://colab.research.google.com/github/JavierHerreraPadilla/Story/blob/master/mindee_facturas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
import requests
import os
from pathlib import Path
import pandas as pd
from numpy import nan 

In [212]:
#%pip install mindee

In [213]:
end_pint = 'https://api.mindee.net/v1/products/mindee/invoices/v4/predict'
api_key = '524cb5c63f2fc3e8abdc91f3699e2d27'
mindee_client = Client(api_key=api_key)

In [214]:
facturas = [Path('./' + element) for element in os.listdir() if Path('./' + element).is_file() and element.lower().endswith('.pdf')]

In [215]:
facturas

[PosixPath('11 11 desayuno.pdf'),
 PosixPath('27 11 cena.pdf'),
 PosixPath('25 11 almuerzo.pdf'),
 PosixPath('27 11 almuerzo.pdf'),
 PosixPath('20 11 cena neiva.pdf'),
 PosixPath('11 11 media tarde.pdf'),
 PosixPath('20 11 desayuno.pdf'),
 PosixPath('20 11 median mañana.pdf'),
 PosixPath('19 11 cena.pdf'),
 PosixPath('5 11 gasolina.pdf'),
 PosixPath('11 almuerzo.pdf'),
 PosixPath('20 11 media tarde.pdf'),
 PosixPath('25 11 media tarde.pdf'),
 PosixPath('27 11 media mañana.pdf'),
 PosixPath('20 11 almuerzo.pdf'),
 PosixPath('11 11 cena.pdf'),
 PosixPath('25 1 cena.pdf'),
 PosixPath('25 11 media mañana.pdf'),
 PosixPath('27 11 desayuno.pdf'),
 PosixPath('11 11 media mañana.pdf')]

In [216]:
invoice_data = list() # lista de la info extraída de cada factura

In [217]:
for factura in facturas:
  input_doc = mindee_client.doc_from_path(factura)

  api_response = input_doc.parse(documents.TypeInvoiceV4)
  invoice_data.append(api_response)


In [218]:
fecha=[]
proveedor=[]
cliente=[]
monto=[]
file_name = []

In [219]:
for fact in invoice_data:
  fecha.append(fact.http_response['document']['inference']['prediction']['date'].get('value', nan))
  proveedor.append(fact.http_response['document']['inference']['prediction']['supplier_name'].get('value', nan))
  cliente.append(fact.http_response['document']['inference']['prediction']['customer_name'].get('value', nan))
  monto.append(fact.http_response['document']['inference']['prediction']['total_amount'].get('value', nan))
  file_name.append(fact.input_filename)

In [220]:
df = pd.DataFrame()
df = pd.DataFrame({'Nombre Archivo':file_name,
                    'Fecha':fecha,
                   'Proveedor':proveedor,
                   'Cliente':cliente,
                   'Monto':monto},
                   )

In [221]:
df

,Nombre Archivo,Fecha,Proveedor,Cliente,Monto
0,11 11 desayuno.pdf,2021-10-01,GRUPO ÉXITO,NCH COLOMBIA S.A.S,12900.0
1,27 11 cena.pdf,None,GRUPO ÉXITO,NCH COLOMBIA SA,22150.0
2,25 11 almuerzo.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A,14850.0
3,27 11 almuerzo.pdf,None,GRUPO ÉXRTO,NCH COLOMBIA SA,13590.0
4,20 11 cena neiva.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A,16250.0
5,11 11 media tarde.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A.S,4180.0
6,20 11 desayuno.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A,NaN
7,20 11 median mañana.pdf,2021-10-01,GRUPO ÉXRTO,None,NaN
8,19 11 cena.pdf,None,None,None,NaN
9,5 11 gasolina.pdf,2021-11-05,SERVICOM ASOCIADOS LTDA,NCH COLOMBIA S A,30001.0


In [222]:
df[~df['Monto'].isna()]['Monto'].sum()

194951.0

In [223]:
df.loc[df.index[-1]+1, ['Nombre Archivo', 'Monto']] = ['Total', df[~df['Monto'].isna()]['Monto'].sum()]
#df.loc[df.index[-1]+1, 'Monto'] = df[~df['Monto'].isna()]['Monto'].sum()

In [224]:
df.to_excel('reembolso.xlsx')

In [225]:
df

,Nombre Archivo,Fecha,Proveedor,Cliente,Monto
0,11 11 desayuno.pdf,2021-10-01,GRUPO ÉXITO,NCH COLOMBIA S.A.S,12900.0
1,27 11 cena.pdf,None,GRUPO ÉXITO,NCH COLOMBIA SA,22150.0
2,25 11 almuerzo.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A,14850.0
3,27 11 almuerzo.pdf,None,GRUPO ÉXRTO,NCH COLOMBIA SA,13590.0
4,20 11 cena neiva.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A,16250.0
5,11 11 media tarde.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A.S,4180.0
6,20 11 desayuno.pdf,2021-10-01,GRUPO ÉXRTO,NCH COLOMBIA S.A,NaN
7,20 11 median mañana.pdf,2021-10-01,GRUPO ÉXRTO,None,NaN
8,19 11 cena.pdf,None,None,None,NaN
9,5 11 gasolina.pdf,2021-11-05,SERVICOM ASOCIADOS LTDA,NCH COLOMBIA S A,30001.0
